# Pydantic BaseModel and Field Lab Tutorial

## Overview
This lab teaches you how to use Pydantic's `BaseModel` and `Field` for data validation, serialization, and creating robust data models in Python. Pydantic is essential for modern Python applications, especially APIs and data processing.




In [17]:
import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")

!pip install openai

# Install Pydantic
!pip install pydantic

# Clean up the notebook
clean_notebook()

Notebook cleaned.


## 🧱 Part 1: Introduction to `BaseModel`

`Pydantic BaseModel` คือ **คลาสหลัก**ที่ใช้ในไลบรารี [Pydantic](https://docs.pydantic.dev) สำหรับ **สร้างโมเดลข้อมูลในภาษา Python**  
โดยมีความสามารถในการ:

- ✅ ตรวจสอบความถูกต้องของข้อมูล (Data Validation)
- 🔄 แปลงชนิดข้อมูลให้อัตโนมัติ (Parsing)

---

### ✏️ อธิบายสั้นๆ:
> `BaseModel` คือแม่แบบสำหรับสร้างคลาสที่มีการกำหนดชนิดข้อมูล (type annotations) อย่างชัดเจน  
> และจะตรวจสอบความถูกต้องของข้อมูลให้ทันทีเมื่อมีการสร้างอ็อบเจกต์

---

### 📦 ตัวอย่างการใช้งานเบื้องต้น:

```python
from pydantic import BaseModel

class User(BaseModel):
    name: str
    age: int

user = User(name="Alice", age=30)        # ✅ ถูกต้อง
user2 = User(name="Bob", age="30")       # ✅ แปลง string เป็น int อัตโนมัติ
user3 = User(name="Tom", age="abc")      # ❌ Error: age ต้องเป็นตัวเลข


### Exercise 1.1: Your First BaseModel
Create a simple user model:

In [18]:
from pydantic import BaseModel
from typing import Optional

class User(BaseModel):
    name: str
    age: int
    email: str
    is_active: bool = True  # Default value

# Create an instance
user = User(name="Alice", age=25, email="alice@example.com")
print(user)
print(f"Name: {user.name}")  # Access attributes
print(f"Dictionary: {user.model_dump()}")  # Convert to dictionary

name='Alice' age=25 email='alice@example.com' is_active=True
Name: Alice
Dictionary: {'name': 'Alice', 'age': 25, 'email': 'alice@example.com', 'is_active': True}


### Exercise 1.2: Automatic Validation
Try creating invalid data and see how Pydantic handles it:

In [19]:
# This will raise a ValidationError
try:
    invalid_user = User(name="Bob", age="not a number", email="bob@example.com")
except ValueError as e:
    print(f"Validation Error: {e}")



Validation Error: 1 validation error for User
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='not a number', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_parsing


In [20]:
# This will work - Pydantic converts string to int
valid_user = User(name="Charlie", age="30", email="charlie@example.com")

print(valid_user)
print(f"Age: {valid_user.age}, Type: {type(valid_user.age)}")

name='Charlie' age=30 email='charlie@example.com' is_active=True
Age: 30, Type: <class 'int'>


## Part 2: Introduction to Field

### What is Field?
Field allows you to add validation rules, default values, descriptions, and constraints to your model attributes.

### Exercise 2.1: Basic Field Usage

In [21]:
from pydantic import BaseModel, Field

class Product(BaseModel):
    name: str = Field(..., min_length=1, max_length=100, description="Product name")
    price: float = Field(..., gt=0, description="Price must be positive")
    quantity: int = Field(default=0, ge=0, description="Quantity in stock")
    category: str = Field("general", description="Product category")

# Create a product
product = Product(name="Laptop", price=999.99, quantity=5, category="electronics")
print(product)
print(f"\nField schema: {Product.model_json_schema()}")

name='Laptop' price=999.99 quantity=5 category='electronics'

Field schema: {'properties': {'name': {'description': 'Product name', 'maxLength': 100, 'minLength': 1, 'title': 'Name', 'type': 'string'}, 'price': {'description': 'Price must be positive', 'exclusiveMinimum': 0.0, 'title': 'Price', 'type': 'number'}, 'quantity': {'default': 0, 'description': 'Quantity in stock', 'minimum': 0, 'title': 'Quantity', 'type': 'integer'}, 'category': {'default': 'general', 'description': 'Product category', 'title': 'Category', 'type': 'string'}}, 'required': ['name', 'price'], 'title': 'Product', 'type': 'object'}


**Field Parameters:**
- `...` (Ellipsis): Required field
- `default`: Default value
- `gt`, `ge`, `lt`, `le`: Greater than, greater/equal, less than, less/equal
- `min_length`, `max_length`: String length constraints
- `description`: Field description for documentation

## Part 3: Practical Examples

### Exercise 3.1: API Response Model
Create a model for handling API responses:

โค้ดนี้ใช้ไลบรารี Pydantic เพื่อสร้าง model สำหรับการตอบกลับ API (API Response) โดยกำหนดชนิดข้อมูลและเงื่อนไขของแต่ละฟิลด์อย่างชัดเจน เพื่อให้มั่นใจว่าโครงสร้างข้อมูลถูกต้องและปลอดภัย



In [22]:
from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import datetime

class APIResponse(BaseModel):
    success: bool = Field(default=True)
    message: str = Field(..., max_length=500)
    data: Optional[dict] = None
    timestamp: datetime = Field(default_factory=datetime.now)
    errors: List[str] = Field(default_factory=list)

# Usage
response = APIResponse(
    message="Data retrieved successfully",
    data={"users": [{"id": 1, "name": "Alice"}]}
)
print(response.model_dump())


{'success': True, 'message': 'Data retrieved successfully', 'data': {'users': [{'id': 1, 'name': 'Alice'}]}, 'timestamp': datetime.datetime(2025, 7, 4, 12, 38, 18, 22385), 'errors': []}


ตัวอย่างการสร้าง error response ด้วยโมเดล APIResponse ที่กำหนดไว้ก่อนหน้า โดยใช้สำหรับตอบกลับในกรณีที่เกิดข้อผิดพลาด (เช่น validation ผิดพลาด)



In [23]:

# Error response
error_response = APIResponse(
    success=False,
    message="Validation failed",
    errors=["Invalid email format", "Password too short"]
)
print("\nError response:")
print(error_response.model_dump())


Error response:
{'success': False, 'message': 'Validation failed', 'data': None, 'timestamp': datetime.datetime(2025, 7, 4, 12, 38, 18, 33262), 'errors': ['Invalid email format', 'Password too short']}


### Exercise 4.: Model Inheritance

โค้ดนี้เป็นการออกแบบ Data Model สำหรับระบบฐานข้อมูลหรือ API โดยใช้ Pydantic เพื่อกำหนดโครงสร้างของ User และ Product โดยมี BaseEntity เป็นคลาสฐานที่แชร์ฟิลด์ร่วมกัน เช่น id, created_at, และ updated_at

In [24]:
from pydantic import BaseModel, Field
from datetime import datetime

class BaseEntity(BaseModel):
    id: int = Field(..., ge=1)
    created_at: datetime = Field(default_factory=datetime.now)
    updated_at: datetime = Field(default_factory=datetime.now)

class User(BaseEntity):
    name: str = Field(..., min_length=1)
    email: str = Field(...)

class Product(BaseEntity):
    name: str = Field(..., min_length=1)
    price: float = Field(..., gt=0)
    category: str = Field(...)

# Usage
user = User(id=1, name="Alice", email="alice@example.com")
product = Product(id=1, name="Laptop", price=999.99, category="Electronics")

print("User:", user)
print("\nProduct:", product)
print(f"\nUser created at: {user.created_at}")
print(f"Product created at: {product.created_at}")

User: id=1 created_at=datetime.datetime(2025, 7, 4, 12, 38, 18, 43734) updated_at=datetime.datetime(2025, 7, 4, 12, 38, 18, 43737) name='Alice' email='alice@example.com'

Product: id=1 created_at=datetime.datetime(2025, 7, 4, 12, 38, 18, 43757) updated_at=datetime.datetime(2025, 7, 4, 12, 38, 18, 43758) name='Laptop' price=999.99 category='Electronics'

User created at: 2025-07-04 12:38:18.043734
Product created at: 2025-07-04 12:38:18.043757


### Exercise 5:   Configuration Design using Pydantic (with field_validator)
Create a configuration model for an application:

โค้ดนี้สร้างโมเดลการตั้งค่าแอปพลิเคชันโดยใช้ Pydantic เพื่อทำ validation, จัดโครงสร้างข้อมูล, และตรวจสอบความถูกต้องของค่า config เช่น ข้อมูลฐานข้อมูล และพารามิเตอร์ทั่วไปของแอป

In [25]:
from pydantic import BaseModel, Field, field_validator
from typing import List

class DatabaseConfig(BaseModel):
    host: str = Field(..., min_length=1)
    port: int = Field(default=5432, ge=1, le=65535)
    database: str = Field(..., min_length=1)
    username: str = Field(..., min_length=1)
    password: str = Field(..., min_length=8)
    
    @field_validator('password')
    def validate_password(cls, v):
        if not any(c.isupper() for c in v):
            raise ValueError('Password must contain uppercase letter')
        if not any(c.islower() for c in v):
            raise ValueError('Password must contain lowercase letter')
        if not any(c.isdigit() for c in v):
            raise ValueError('Password must contain digit')
        return v

class AppConfig(BaseModel):
    app_name: str = Field(..., min_length=1)
    debug: bool = Field(default=False)
    allowed_hosts: List[str] = Field(default_factory=list)
    database: DatabaseConfig
    max_connections: int = Field(default=100, ge=1, le=1000)

# Usage
config = AppConfig(
    app_name="MyApp",
    database=DatabaseConfig(
        host="localhost",
        database="myapp_db",
        username="admin",
        password="SecurePass123"
    ),
    allowed_hosts=["localhost", "127.0.0.1"]
)
print(config.model_dump())

{'app_name': 'MyApp', 'debug': False, 'allowed_hosts': ['localhost', '127.0.0.1'], 'database': {'host': 'localhost', 'port': 5432, 'database': 'myapp_db', 'username': 'admin', 'password': 'SecurePass123'}, 'max_connections': 100}


## Part 5: LAB computed fields อัตโนมัติ

###  E-commerce Order System
โค้ดนี้ใช้ pydantic เพื่อสร้างโมเดลข้อมูลของระบบจัดการคำสั่งซื้อ (Order Management System) โดยมีการตรวจสอบความถูกต้องของข้อมูล (validation) และคำนวณค่ารวม (computed fields) อัตโนมัติ

In [26]:
from pydantic import BaseModel, Field, field_validator, computed_field
from typing import List, Optional
from datetime import datetime
from enum import Enum

class OrderStatus(str, Enum):
    PENDING = "pending"
    CONFIRMED = "confirmed"
    SHIPPED = "shipped"
    DELIVERED = "delivered"
    CANCELLED = "cancelled"

class OrderItem(BaseModel):
    product_id: int = Field(..., ge=1)
    product_name: str = Field(..., min_length=1)
    quantity: int = Field(..., ge=1)
    unit_price: float = Field(..., gt=0)
    
    @computed_field
    @property
    def total_price(self) -> float:
        return self.quantity * self.unit_price

class Order(BaseModel):
    order_id: str = Field(..., pattern=r'^ORD\d{6}$')
    customer_email: str = Field(...)
    items: List[OrderItem] = Field(..., min_length=1)
    status: OrderStatus = Field(default=OrderStatus.PENDING)
    order_date: datetime = Field(default_factory=datetime.now)
    shipping_address: str = Field(..., min_length=10)
    
    @field_validator('customer_email')
    def validate_email(cls, v):
        if '@' not in v or '.' not in v:
            raise ValueError('Invalid email format')
        return v.lower()
    
    @computed_field
    @property
    def total_amount(self) -> float:
        return sum(item.total_price for item in self.items)


In [27]:

# Test your implementation
order = Order(
    order_id="ORD123456",
    customer_email="customer@example.com",
    items=[
        OrderItem(product_id=1, product_name="Laptop", quantity=1, unit_price=999.99),
        OrderItem(product_id=2, product_name="Mouse", quantity=2, unit_price=25.99)
    ],
    shipping_address="123 Main St, City, State 12345"
)


In [28]:
print(order)

order_id='ORD123456' customer_email='customer@example.com' items=[OrderItem(product_id=1, product_name='Laptop', quantity=1, unit_price=999.99, total_price=999.99), OrderItem(product_id=2, product_name='Mouse', quantity=2, unit_price=25.99, total_price=51.98)] status=<OrderStatus.PENDING: 'pending'> order_date=datetime.datetime(2025, 7, 4, 12, 38, 18, 63034) shipping_address='123 Main St, City, State 12345' total_amount=1051.97


In [29]:
print(f"Order Total: ${order.total_amount:.2f}")

Order Total: $1051.97


In [30]:

print(f"Order Total: ${order.total_amount:.2f}")
print(f"\nOrder Details:")
for item in order.items:
    print(f"- {item.product_name}: {item.quantity} x ${item.unit_price} = ${item.total_price}")
    
print(f"\nFull Order: {order.model_dump()}")

Order Total: $1051.97

Order Details:
- Laptop: 1 x $999.99 = $999.99
- Mouse: 2 x $25.99 = $51.98

Full Order: {'order_id': 'ORD123456', 'customer_email': 'customer@example.com', 'items': [{'product_id': 1, 'product_name': 'Laptop', 'quantity': 1, 'unit_price': 999.99, 'total_price': 999.99}, {'product_id': 2, 'product_name': 'Mouse', 'quantity': 2, 'unit_price': 25.99, 'total_price': 51.98}], 'status': <OrderStatus.PENDING: 'pending'>, 'order_date': datetime.datetime(2025, 7, 4, 12, 38, 18, 63034), 'shipping_address': '123 Main St, City, State 12345', 'total_amount': 1051.97}


### Challenge 2: Student Grade Management
Create a system to manage student grades:

In [31]:
from pydantic import BaseModel, Field, field_validator, computed_field
from typing import List, Dict, Optional
from datetime import datetime

class Grade(BaseModel):
    subject: str = Field(..., min_length=1)
    score: float = Field(..., ge=0, le=100)
    max_score: float = Field(default=100, gt=0)
    date: datetime = Field(default_factory=datetime.now)
    
    @computed_field
    @property
    def percentage(self) -> float:
        return (self.score / self.max_score) * 100
    
    @computed_field
    @property
    def letter_grade(self) -> str:
        pct = self.percentage
        if pct >= 90: return "A"
        elif pct >= 80: return "B"
        elif pct >= 70: return "C"
        elif pct >= 60: return "D"
        else: return "F"

class Student(BaseModel):
    student_id: str = Field(..., pattern=r'^STU\d{6}$')
    name: str = Field(..., min_length=1)
    email: str = Field(...)
    grades: List[Grade] = Field(default_factory=list)
    
    @field_validator('email')
    def validate_email(cls, v):
        if '@' not in v:
            raise ValueError('Invalid email format')
        return v.lower()
    
    def add_grade(self, grade: Grade):
        self.grades.append(grade)
    
    @computed_field
    @property
    def gpa(self) -> float:
        if not self.grades:
            return 0.0
        
        grade_points = {"A": 4.0, "B": 3.0, "C": 2.0, "D": 1.0, "F": 0.0}
        total_points = sum(grade_points[grade.letter_grade] for grade in self.grades)
        return total_points / len(self.grades)


In [32]:

# Test your implementation
student = Student(
    student_id="STU123456",
    name="Alice Johnson",
    email="alice.johnson@school.edu"
)

student.add_grade(Grade(subject="Mathematics", score=85))
student.add_grade(Grade(subject="Science", score=92))
student.add_grade(Grade(subject="English", score=78))


In [33]:

print(f"Student: {student.name}")
print(f"GPA: {student.gpa:.2f}")
print("\nGrades:")
for grade in student.grades:
    print(f"- {grade.subject}: {grade.score}/100 ({grade.letter_grade}) - {grade.percentage:.1f}%")

print(f"\nFull Student Record: {student.model_dump()}")

Student: Alice Johnson
GPA: 3.00

Grades:
- Mathematics: 85.0/100 (B) - 85.0%
- Science: 92.0/100 (A) - 92.0%
- English: 78.0/100 (C) - 78.0%

Full Student Record: {'student_id': 'STU123456', 'name': 'Alice Johnson', 'email': 'alice.johnson@school.edu', 'grades': [{'subject': 'Mathematics', 'score': 85.0, 'max_score': 100, 'date': datetime.datetime(2025, 7, 4, 12, 38, 18, 113273), 'percentage': 85.0, 'letter_grade': 'B'}, {'subject': 'Science', 'score': 92.0, 'max_score': 100, 'date': datetime.datetime(2025, 7, 4, 12, 38, 18, 113290), 'percentage': 92.0, 'letter_grade': 'A'}, {'subject': 'English', 'score': 78.0, 'max_score': 100, 'date': datetime.datetime(2025, 7, 4, 12, 38, 18, 113306), 'percentage': 78.0, 'letter_grade': 'C'}], 'gpa': 3.0}
